# Functions

Topics:
1. How to declare a function
2. Duck-typing in Julia
3. Mutating vs. non-mutating functions
4. Some higher order functions

## How to declare a function
Julia gives us a few different ways to write a function. The first requires the `function` and `end` keywords

In [1]:
function name(var)
    #body
end

name (generic function with 1 method)

We can call either of these functions like this:

In [2]:
name(3)

Alternatively, we could have declared either of these functions in a single line

In [3]:
f(x)=2*x

f (generic function with 1 method)

In [4]:
f(3)

6

Finally, we could have declared these as "anonymous" functions

In [6]:
g=(x) -> 2*x

#5 (generic function with 1 method)

In [8]:
map(x->2*x,[1,2,3])

3-element Array{Int64,1}:
 2
 4
 6

## Duck-typing in Julia
*"If it quacks like a duck, it's a duck."* <br><br>
Julia functions will just work on whatever inputs make sense. <br><br>
For example, `sayhi` works on the name of this minor tv character, written as an integer...

And `f` will work on a matrix. 

In [14]:
f(x)=x^2
@code_llvm f(2)



; Function f
; Location: In[14]:1
define i64 @julia_f_35986(i64) {
top:
; Function literal_pow; {
; Location: intfuncs.jl:243
; Function *; {
; Location: int.jl:54
  %1 = mul i64 %0, %0
;}}
  ret i64 %1
}


In [15]:
methods(f)

# 1 method for generic function "f":
[1] f(x) in Main at In[14]:1

`f` will also work on a string like "hi" because `*` is defined for string inputs as string concatenation.

On the other hand, `f` will not work on a vector. Unlike `A^2`, which is well-defined, the meaning of `v^2` for a vector, `v`, is not a well-defined algebraic operation. 

In [16]:
f([2,3,4])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:795
  ^(!Matched::Missing, ::Integer) at missing.jl:120
  ^(!Matched::Missing, ::Number) at missing.jl:93
  ...

## Mutating vs. non-mutating functions

By convention, functions followed by `!` alter their contents and functions lacking `!` do not.

For example, let's look at the difference between `sort` and `sort!`.


In [19]:
a=[1,4,0,2];
sort!(a)
a

4-element Array{Int64,1}:
 0
 1
 2
 4

`sort(v)` returns a sorted array that contains the same elements as `v`, but `v` is left unchanged. <br><br>

On the other hand, when we run `sort!(v)`, the contents of v are sorted within the array `v`.

In [20]:
f(x::Int)=x+x

f (generic function with 2 methods)

## Some higher order functions

### map

`map` is a "higher-order" function in Julia that *takes a function* as one of its input arguments. 
`map` then applies that function to every element of the data structure you pass it. For example, executing

```julia
map(f, [1, 2, 3])
```
will give you an output array where the function `f` has been applied to all elements of `[1, 2, 3]`
```julia
[f(1), f(2), f(3)]
```

In [21]:
map(f, [1, 2, 3])

3-element Array{Int64,1}:
 2
 4
 6

In [23]:
@code_native f(2)

	.text
; Function f {
; Location: In[20]:1
; Function +; {
; Location: In[20]:1
	leaq	(%rdi,%rdi), %rax
;}
	retq
	nopw	%cs:(%rax,%rax)
;}


Here we've squared all the elements of the vector `[1, 2, 3]`, rather than squaring the vector `[1, 2, 3]`.

To do this, we could have passed to `map` an anonymous function rather than a named function, such as

via

and now we've cubed all the elements of `[1, 2, 3]`!

### broadcast

`broadcast` is another higher-order function like `map`. `broadcast` is a generalization of `map`, so it can do every thing `map` can do and more. The syntax for calling `broadcast` is the same as for calling `map`

and again, we've applied `f` (squared) to all the elements of `[1, 2, 3]` - this time by "broadcasting" `f`!

Some syntactic sugar for calling `broadcast` is to place a `.` between the name of the function you want to `broadcast` and its input arguments. For example,

```julia
broadcast(f, [1, 2, 3])
```
is the same as
```julia
f.([1, 2, 3])
```

In [24]:
broadcast(f,[1,2,3])

3-element Array{Int64,1}:
 2
 4
 6

In [25]:
f.([1, 2, 3])

3-element Array{Int64,1}:
 2
 4
 6

Notice again how different this is from calling 
```julia
f([1, 2, 3])
```
We can square every element of a vector, but we can't square a vector!

In [26]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:795
  ^(!Matched::Missing, ::Integer) at missing.jl:120
  ^(!Matched::Missing, ::Number) at missing.jl:93
  ...

To drive home the point, let's look at the difference between

```julia
f(A)
```
and
```julia
f.(A)
```
for a matrix `A`:

In [28]:
A=rand(4,4)
@show f(A)
f.(A)

f(A) = [1.47726 0.797463 1.12983 0.672613; 2.07719 1.19767 1.69121 1.05761; 2.24762 1.19003 1.68951 0.964699; 1.89103 1.02851 1.45667 0.87732]


4×4 Array{Float64,2}:
 0.571647  0.114164  0.232186  0.0566249
 0.474387  0.468634  0.854542  0.0564042
 0.868661  0.176312  0.38228   0.518926 
 0.884495  0.207391  0.425638  0.0753951

As before we see that for a matrix, `A`,
```
f(A) = A^2 = A * A
``` 

On the other hand,

In [34]:
@. A + 1 + 3 

4×4 Array{Float64,2}:
 4.75607  4.33788  4.48186  4.23796
 4.68876  4.68457  4.92441  4.2375 
 4.93202  4.41989  4.61829  4.72037
 4.94048  4.4554   4.65241  4.27458

contains the squares of all the entries of `A`.

This dot syntax for broadcasting allows us to write relatively complex compound elementwise expressions in a way that looks natural/closer to mathematical notation. For example, we can write

instead of

and the two will perform exactly the same.

### Exercises

#### 6.1 
Write a function `add_one` that adds 1 to its input.

#### 6.2 
Use `map` or `broadcast` to increment every element of matrix `A` by `1` and assign it to a variable `A1`.

#### 6.3 
Use the broadcast dot syntax to increment every element of matrix `A1` by `1` and store it in variable `A2`

Please click on `Validate` on the top, once you are done with the exercises.